# AZURE SPEECH SDK PRESENTATION

In [1]:
# on commence par installer le SDK: pip install azure-cognitiveservices-speech

import azure.cognitiveservices.speech as speechsdk
from config import key, location
import time

La [documentation/tutp](https://docs.microsoft.com/fr-fr/azure/cognitive-services/speech-service/)

La [documentation/détail des objets](https://docs.microsoft.com/en-us/python/api/azure-cognitiveservices-speech/azure.cognitiveservices.speech?view=azure-python)

## Azure Speech to Text (via le SDK)

Il existe trois moyen d'utiliser speech to text:
- via le [CLI](https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/spx-overview) (no code, simple usage)
- via le [SDK](https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/speech-sdk?tabs=windows%2Cubuntu%2Cios-xcode%2Cmac-xcode%2Candroid-studio) (à privilégier)
- via les [API REST](https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/rest-speech-to-text)
  - Speech-to-text REST API v3.0 is used for Batch transcription and Custom Speech.
  - Speech-to-text REST API for short audio is used for online transcription (comme transcription continue mais limité à 60 seconde)

### Les objets du SDK speech-to-text

le Kit de développement logiciel (SDK) possède 4 objets principaux:

![speech to text](https://docs.microsoft.com/en-us/learn/wwl-data-ai/transcribe-speech-input-text/media/speech-to-text.png)

1. SpeechConfig pour encapsuler les informations de connexion à votre ressource Speech

In [2]:
speech_config = speechsdk.SpeechConfig(subscription=key, region=location)

2. AudioConfig pour définir la source d’entrée de l’audio à transcrire.

In [3]:
# format should be .wav or .ogg
audio_input = speechsdk.AudioConfig(filename="absinthe.wav")

3. SpeechRecognizer: le client proxy pour l’API de reconnaissance vocale.

In [4]:
# It's here you change the recognition source.
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config,language="fr-FR", audio_config=audio_input)

**`Microphone`**

Si dans le speech_recognizer aucun audioConfig n'est précisée alors par défaut la source audio est votre microphone(si vous avez plusieurs microphone vous pouvez [indiquer dans un audioconfig](https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/get-started-speech-to-text?tabs=windowsinstall&pivots=programming-language-python) l'id de votre microphone)

4 Le dernier objet important est le retour obtenu mais il dépend de la méthode appelée.

### Les méthodes de speechrecognizer

#### recognize_once() et recognize_once_async() : Reconnaitre un mot

Returns after a single utterance is recognized. The end of a single utterance is determined by listening for silence at the end or until a maximum of 15 seconds of audio is processed.

Comme toujours, la méthode avec async permet de ne pas arreter l'execution code si l'opération prend du temps.

In [ ]:
result = speech_recognizer.recognize_once_async().get()

Le retour est un [SpeechRecognitionResult](https://docs.microsoft.com/en-us/javascript/api/microsoft-cognitiveservices-speech-sdk/speechrecognitionresult?view=azure-node-latest)
Il possède les attributs suivants:
- reason
  - RecognizedSpeech: la reconnaissance a fonctionné
  - NoMatch: la reconnaissance a fonctionné mais rien n'a pu être reconnu
  - Canceled: erreur ou annulation
    - il y un object: result.cancellation_details.raison ..
- Duration: Duration of recognized speech
- Text: résultat de la reconnaissance.
- language: Primary Language detected.

In [ ]:
with open('file_name.txt', 'w') as f:
    f.write(result.text)
print("done")


#### start_continuous_recognition() reconnaitre un texte 

La reconnaissance continue nécessite 4 étapes:
- le set up 
- le lancement de la reconnaissance
- le traitement des événements
- l'arret de la reconnaissance;

Avant de commencer il faut comprendre une chose:
Quand une session de reconnaissance continue est lancée, cette opération prend du temps. Au cours de celle-ci des événements sont envoyés à l'EventSignal. Il est possible de se connecter à cet event Signal pour recevoir tous les signals correspondant.

L'objet speech_recognizer possède plusieurs attributs dont il sera possible d'obtenir les événements:
- `recognizing`: Signal for events containing intermediate recognition results.
- `recognized`: Signal for events containing final recognition results (indicating a successful recognition attempt).
- `session_started`: Signal for events indicating the start of a recognition session (operation).
- `session_stopped`: Signal for events indicating the end of a recognition session (operation).
- `canceled`: Signal for events containing canceled recognition results (indicating a recognition attempt that was canceled as a result or a direct cancellation request or, alternatively, a transport or protocol failure).

Par exemple pour l'événement recognized. On peut créer un client pour détecter ces événements à l'aide du code ci-dessous

In [ ]:
speech_recognizer.recognized.connect("function name")

Pour chaque événement détecter on lui fait passer une fonction qui peut avoir plusieurs but:
- afficher l'événement
- enregistrer la sortie de l'événement(le texte reconnu)
- arreter la reconnaissance.

On peut à présent définir le setup

In [ ]:
# on définit les fonctions qui vont être utiles
# une pour arreter la reconnaissance
def stop_cb(evt):
    """callback that stops continuous recognition upon receiving an event `evt`"""
    print('CLOSING on {}'.format(evt))
    speech_recognizer.stop_continuous_recognition()

# une pour enregistrer le résultat
all_results = []
def handle_final_result(evt):
    all_results.append(evt.result.text)

# on initilise ensuite nos clients et on leur fait passer les fonctions voulues
# le client lié au début de session
speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))

# les clients liées à la reconnaissance du texte
speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))

speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
speech_recognizer.recognized.connect(handle_final_result)

# les clients liées à la fin de session
speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
speech_recognizer.session_stopped.connect(stop_cb)

speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
speech_recognizer.canceled.connect(stop_cb)

# On peut ensuite lancer la reconnaissance.
speech_recognizer.start_continuous_recognition()

#affiche le résultat
#print(all_results)

### Options supplémentaires

#### Dictation mode

This mode will cause the speech config instance to interpret word descriptions of sentence structures such as punctuation. 

For example, the utterance "Do you live in town question mark" would be interpreted as the text "Do you live in town?".

To enable dictation mode, use the enable_dictation() method on your SpeechConfig.

In [ ]:
SpeechConfig.enable_dictation()

#### Phrase list

On peut donner une liste de phrase qu'on sait être dans le texte pour aider la reconnaissance vocale.

`Par exemple` si on sait quand dans le texte on a "Move to Ward" on peut ajouter cette phrase à la liste pour éviter que azure comprenne "Move toward"

In [ ]:
# on crée  pour cela un objet phraselistgrammar
phrase_list_grammar = speechsdk.PhraseListGrammar.from_recognizer(speech_recognizer)
phrase_list_grammar.addPhrase("Move to Ward")

In [ ]:
# l'objet possède également une méthode clear
phrase_list_grammar.clear()